# Training and Evaluation for CINIC10 using SimpleCNN

This notebook loads the converted dataset, trains the SimpleCNN model, and performs evaluation and inference on the validation set.

In [16]:
import os
import torch
import torch.optim as optim
import torch.nn as nn

from DataObjects import DataLoader
from Architectures.SimpleCNN import SimpleCNN
from Architectures.OptimalCNN import OptimalCNN

from typing import Optional, Tuple

In [17]:
def train_model(model: nn.Module, train_loader: DataLoader, val_loader: DataLoader,
                num_epochs: int = 10, lr: float = 0.001,
                device: torch.device = None) -> None:
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion: nn.Module = nn.CrossEntropyLoss()
    optimizer: torch.optim.Optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(num_epochs):
        model.train()
        train_loss: float = 0.0
        train_correct: int = 0
        total_train: int = 0
        
        for batch in train_loader:
            inputs = batch.data.to(device)
            labels = batch.labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels).item()
            total_train += labels.size(0)
        
        avg_train_loss = train_loss / total_train
        train_acc = train_correct / total_train
        
        model.eval()
        val_loss: float = 0.0
        val_correct: int = 0
        total_val: int = 0
        
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch.data.to(device)
                labels = batch.labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels).item()
                total_val += labels.size(0)
        
        avg_val_loss = val_loss / total_val
        val_acc = val_correct / total_val
        
        print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {avg_train_loss:.4f}, Train acc: {train_acc:.4f} | Val loss: {avg_val_loss:.4f}, Val acc: {val_acc:.4f}")

def infer(model: nn.Module, data_loader: DataLoader,
          device: torch.device = None) -> list:
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    predictions: list = []
    
    with torch.no_grad():
        for batch in data_loader:
            inputs = batch.data.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().tolist())
    
    return predictions


def evaluate(model: nn.Module,
             test_loader: Optional[DataLoader] = None,
             device: Optional[torch.device] = None) -> Tuple[float, float]:

    if test_loader is None:
        test_dir = os.path.join("Data", "Data_converted", "test")
        test_loader = DataLoader(test_dir, batch_size=64, shuffle=True)
        
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    model.eval()
    
    test_loss = 0.0
    test_correct = 0
    total_test = 0
    
    with torch.no_grad():
        for batch in test_loader:
            inputs = batch.data.to(device)
            labels = batch.labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            test_correct += torch.sum(preds == labels).item()
            total_test += labels.size(0)
    
    avg_test_loss = test_loss / total_test
    test_acc = test_correct / total_test
    
    print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
    return avg_test_loss, test_acc


In [3]:
# Setup directories and DataLoaders
train_dir = os.path.join("Data", "Data_converted", "train")
val_dir = os.path.join("Data", "Data_converted", "valid")

train_loader = DataLoader(train_dir, batch_size=64, shuffle=True, max_per_class=150)
val_loader = DataLoader(val_dir, batch_size=64, shuffle=False, max_per_class=150) 

# Instantiate the model
model = SimpleCNN(num_classes=10)

In [7]:
# Train the model
train_model(model, train_loader, val_loader, num_epochs=10, lr=0.001)

Epoch 1/10 - Train loss: 1.1707, Train acc: 0.5867 | Val loss: 2.4186, Val acc: 0.2927
Epoch 2/10 - Train loss: 0.8279, Train acc: 0.7253 | Val loss: 2.6018, Val acc: 0.3093
Epoch 3/10 - Train loss: 0.5913, Train acc: 0.8080 | Val loss: 3.0063, Val acc: 0.3160
Epoch 4/10 - Train loss: 0.4302, Train acc: 0.8573 | Val loss: 3.2185, Val acc: 0.3240
Epoch 5/10 - Train loss: 0.3442, Train acc: 0.8920 | Val loss: 3.5537, Val acc: 0.3320
Epoch 6/10 - Train loss: 0.2194, Train acc: 0.9400 | Val loss: 3.8268, Val acc: 0.3293
Epoch 7/10 - Train loss: 0.1430, Train acc: 0.9693 | Val loss: 4.4267, Val acc: 0.3273
Epoch 8/10 - Train loss: 0.1244, Train acc: 0.9687 | Val loss: 4.5080, Val acc: 0.3320
Epoch 9/10 - Train loss: 0.1471, Train acc: 0.9607 | Val loss: 4.5738, Val acc: 0.3327
Epoch 10/10 - Train loss: 0.2647, Train acc: 0.9227 | Val loss: 4.4066, Val acc: 0.3353


In [8]:
test_dir = os.path.join("Data", "Data_converted", "test")
test_loader = DataLoader(test_dir, batch_size=64, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()

model.eval()
test_loss = 0.0
test_correct = 0
total_test = 0

with torch.no_grad():
    for batch in test_loader:
        inputs = batch.data.to(device)
        labels = batch.labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        test_correct += torch.sum(preds == labels).item()
        total_test += labels.size(0)

avg_test_loss = test_loss / total_test
test_acc = test_correct / total_test

print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Test Loss: 5.6793, Test Accuracy: 0.2764


In [9]:
# Full training
train_loader_full = DataLoader(train_dir, batch_size=64, shuffle=True)
val_loader_full = DataLoader(val_dir, batch_size=64, shuffle=False) 

# Instantiate the model
model_2 = SimpleCNN(num_classes=10)

In [11]:
# Train the model
train_model(model_2, train_loader, val_loader, num_epochs=10, lr=0.001)

Epoch 1/10 - Train loss: 0.4291, Train acc: 0.8567 | Val loss: 3.3108, Val acc: 0.3347
Epoch 2/10 - Train loss: 0.2400, Train acc: 0.9373 | Val loss: 3.4594, Val acc: 0.3313
Epoch 3/10 - Train loss: 0.1584, Train acc: 0.9600 | Val loss: 3.8230, Val acc: 0.3373
Epoch 4/10 - Train loss: 0.1177, Train acc: 0.9707 | Val loss: 4.3002, Val acc: 0.3387
Epoch 5/10 - Train loss: 0.0707, Train acc: 0.9880 | Val loss: 4.4719, Val acc: 0.3520
Epoch 6/10 - Train loss: 0.0504, Train acc: 0.9920 | Val loss: 4.7958, Val acc: 0.3433
Epoch 7/10 - Train loss: 0.0460, Train acc: 0.9900 | Val loss: 5.1337, Val acc: 0.3320
Epoch 8/10 - Train loss: 0.0937, Train acc: 0.9740 | Val loss: 5.1910, Val acc: 0.3100
Epoch 9/10 - Train loss: 0.1296, Train acc: 0.9627 | Val loss: 5.4133, Val acc: 0.3193
Epoch 10/10 - Train loss: 0.1153, Train acc: 0.9673 | Val loss: 5.2266, Val acc: 0.3307


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_2.to(device)

criterion = nn.CrossEntropyLoss()

model_2.eval()
test_loss = 0.0
test_correct = 0
total_test = 0

with torch.no_grad():
    for batch in test_loader:
        inputs = batch.data.to(device)
        labels = batch.labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        test_correct += torch.sum(preds == labels).item()
        total_test += labels.size(0)

avg_test_loss = test_loss / total_test
test_acc = test_correct / total_test

print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Test Loss: 5.6793, Test Accuracy: 0.2764


In [18]:
## Different architecture
model_Optimal = OptimalCNN(num_classes=10)

In [ ]:
# Train the model
train_model(model_Optimal, train_loader_full, val_loader_full, num_epochs=10, lr=0.001)

In [ ]:
# Evaluate
evaluate(model_Optimal, test_loader)